### <다중회귀모델(Multiple Regression)>

- 여러 개의 특성을 사용한 회귀모델

- 특성이 많을수록 집중력과 복잡도가 높아짐(단, 훈련 시간이 오래 걸릴 수 있음, 시스템 성능에 따라 처리 시간이 결정됨)

- 다중회귀모델 : $ y = a*x1 + b*x2 +c*x3 + ... + n*x_n + y절편 $

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import pandas as pd

### 시각화 시 한글 및 마이너스(-) 기호 사용을 위한 처리
# - 한글 처리 : 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'

# - 마이너스(-) 기호 처
plt.rcParams['axes.unicode_minus'] =False

In [2]:
### 01_농어의_길이_높이_두께_데이터.csv파일 읽어 들이기 
# - 데이터프레임 변수 이름 : df

df = pd.read_csv('./data/01_농어의_길이_높이_두께_데이터.csv')

### 간단히 결측치 확인
df.info()

### 간단히 이상치 확인 : 기초통계 함수
 
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   length  56 non-null     float64
 1   height  56 non-null     float64
 2   width   56 non-null     float64
dtypes: float64(3)
memory usage: 1.4 KB


,length,height,width
count,56.000000,56.000000,56.000000
mean,27.892857,7.862143,4.745536
std,9.021668,2.878343,1.775006
min,8.400000,2.110000,1.410000
25%,21.825000,5.690000,3.520000
50%,25.300000,6.920000,4.155000
75%,36.625000,10.850000,6.450000
max,44.000000,12.800000,8.140000


### 분석 주제 : 농어의 길이, 두께, 너비를 이용하여 -> 무게 예측하기

- 독립 변수 : 길이, 두께, 너비

- 종속 변수 : 무게

In [3]:
### 독립변수 만들기
### 데이터 프레임 타입의 독립변수를 2차원의 numpy array로 변환하기
# - 왜 바꿔야 하는가 ? : 모델 훈련 시 2차원 배열 또는 리스트를 사용하기 때문에

perch_full = df.to_numpy()
perch_full.shape

(56, 3)

In [4]:
### 종속변수 만들기
# 농어 무게
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )
perch_weight.shape

(56,)

In [5]:
### 훈련:테스트 = 7:3으로 분류하기
# - 사용변수 : train_input, test_input, train_target, test_target
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, test_size=0.3,random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape,test_target.shape)

(39, 3) (39,)
(17, 3) (17,)


In [6]:
### 훈련 모델 생성
lr = LinearRegression()

### 훈련시키기
lr.fit(train_input, train_target)

### 훈련 및 테스트 정확도 확인하기
train_score = lr.score(train_input, train_target)
test_score = lr.score(test_input, test_target)
print(train_score, test_score)

0.9537065271284176 0.8863420836347778


#### 해석

- 농어의 길이, 두께, 너비 특성을 활용하여 농어의 무게를 예측하는 다항회귀모델을 만들었다.

- 훈련 정확도가 0.95로 괜찮은 성능을 보이고 있고, 훈련 정확도가 테스트 정확도보다 크고 0.1보다 작은 차이를 보이기 때문에 과대적합이라고 보기 어렵다.

- 과대적합이 일어나지 않았기 때문에 일반화된 모델이라고 판단된다.

In [7]:
### 테스트 데이터로 예측하기
pred = lr.predict(test_input)
print(pred)

[-328.54944233   48.99648009  328.530777    168.01071374  147.28466932
  790.89418753  383.33837613  247.89949679  814.06695812  123.2000108
  981.8543067   -37.63667416  356.2229673   419.56345249   48.15564696
  140.06819759   46.70641917]


In [8]:
### 평균절대오차(MAE) 확인하기
mae = mean_absolute_error(test_target, pred)
mae

68.56651514640546

#### 해석

- 해당 모델로 예측했을 때 기본적으로 +- 68.57 정도의 오차가 발생할 수 있는 모델로 판단

#####  좋은 모델임에도 불구하고, 성능향상을 더 할 수 있는 방법을 적용해본다.

- 특성공학을 적용하여 특성을 증가시킨다(특성, 즉 컬럼을 추가한다).

- 집중력(복잡도)를 강하게 하는 방법임.

### <특성을 생성하는 라이브러리>

- 사용 패키지 : sklearn.preprocessing

- 사용모델(클래스) : 특성을 생성하는 모델(훈련 모델은 아님), PolynomialFeatures() -> "변환기"라고 칭함

- 사용함수 : fit() -> 훈련 독립변수를 이용하여 특성의 패턴을 찾는다. / transform() -> 찾은 특성의 패턴을 이용하여 특성을 생성한다.

- 종속변수는 절대 사용하거나 변경하면 안 된다. 독립변수만 사용된다.

In [9]:
### 클래스 생성하기
# include_bias : 해당 클래스가 특성을 추가 시킬 때 y절편값도 생성을 하기 때문에 우리는 특성만 필요하다. 
# 따라서 y절편은 제외한다라는 의미로 include_bias =False로 설정 (y절편 생성 여부 설정 속성)
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)

In [10]:
### 예제 데이터로 작동방식 확인하기
temp_data = [[2,3,4]]

### 독립변수의 특성들을 이용해서 패턴 찾기
poly.fit(temp_data)

### 찾은 패턴으로 특성 추가시키기
poly.transform(temp_data)

array([[ 2.,  3.,  4.,  4.,  6.,  8.,  9., 12., 16.]])

#### 특성 패턴

1. 사용된 독립변수에 처음에 그대로 제시됨(3개 특성 제시)

2. 첫번째 값을 제곱한 값

3. 첫번째 값과 두번째 값 곱한 값

4. 첫번째 값과 세번쨰 값 곱한 값

5. 두번째 값을 제곱한 값

6. 두번째 값과 세번째 값 곱한 값

7. 세번째 값을 제곱한 값

In [11]:
### 차원(degree)을 이용하여 패턴 만들어서 특성 추가하기 -> degree는 4이상 사용하지 않음 / default = 2
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(temp_data)

array([[ 2.,  3.,  4.,  4.,  6.,  8.,  9., 12., 16.,  8., 12., 16., 18.,
        24., 32., 27., 36., 48., 64.]])

In [12]:
### 패턴 확인하는 함수 이용하기
poly.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2', 'x0^3', 'x0^2 x1', 'x0^2 x2', 'x0 x1^2', 'x0 x1 x2',
       'x0 x2^2', 'x1^3', 'x1^2 x2', 'x1 x2^2', 'x2^3'], dtype=object)

In [13]:
### --------------------------------------------------
### 실제 독립변수를 이용해서 특성 생성하기
### 기존 특성의 패턴을 찾아서 새로운 특성을 만들어내는 클래스(변환기라는 이름을 붙임)

poly = PolynomialFeatures(degree=2, include_bias=False)
poly

PolynomialFeatures(include_bias=False)

In [14]:
### 패턴 찾기
# - 패턴을 찾을 때는 항상 훈련 독립 변수만 사용한다.
# - 테스트 데이터는 사용하지 않는다.

poly.fit(train_input)

PolynomialFeatures(include_bias=False)

In [15]:
### 새로운 특성 생성하기
### - 특성을 생성할 때는 훈련 및 테스트 독립변수 모두 적용한다.

# 훈련 독립변수에 특성 생성하기
train_poly = poly.transform(train_input)

# 테스트 독립변수에 특성 생성하기
test_poly = poly.transform(test_input)

train_poly.shape, test_poly.shape

((39, 9), (17, 9))

In [16]:
train_poly
test_poly

array([[8.400000e+00, 2.110000e+00, 1.410000e+00, 7.056000e+01,
        1.772400e+01, 1.184400e+01, 4.452100e+00, 2.975100e+00,
        1.988100e+00],
       [1.800000e+01, 5.220000e+00, 3.320000e+00, 3.240000e+02,
        9.396000e+01, 5.976000e+01, 2.724840e+01, 1.733040e+01,
        1.102240e+01],
       [2.750000e+01, 7.280000e+00, 4.570000e+00, 7.562500e+02,
        2.002000e+02, 1.256750e+02, 5.299840e+01, 3.326960e+01,
        2.088490e+01],
       [2.130000e+01, 6.380000e+00, 3.530000e+00, 4.536900e+02,
        1.358940e+02, 7.518900e+01, 4.070440e+01, 2.252140e+01,
        1.246090e+01],
       [2.250000e+01, 5.860000e+00, 3.620000e+00, 5.062500e+02,
        1.318500e+02, 8.145000e+01, 3.433960e+01, 2.121320e+01,
        1.310440e+01],
       [4.000000e+01, 1.114000e+01, 6.630000e+00, 1.600000e+03,
        4.456000e+02, 2.652000e+02, 1.240996e+02, 7.385820e+01,
        4.395690e+01],
       [3.000000e+01, 7.620000e+00, 4.770000e+00, 9.000000e+02,
        2.286000e+02, 1.431000

In [17]:
### 특성 생성에 사용된 패턴 확인하기
poly.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [18]:
### 모델 생성하기
lr = LinearRegression()

### 모델 훈련시키기
lr.fit(train_poly, train_target)

### 훈련 및 테스트 정확도 확인하기
train_score = lr.score(train_poly,train_target)
test_score = lr.score(test_poly,test_target)

print(train_score, test_score)
print(train_score - test_score)

0.9898271546307026 0.9713771600629739
0.018449994567728667


#### 해석 팁

- 과소/과대 적합이 있다면, -> 모델 튜닝 -> 모델 훈련 -> 정확도 확인을 반복하면서  가장 일반화된 모델이 되는 시점을 찾아야 함


- 일반화된 모델을 찾은 후 아래 예측으로 넘어간다.

#### 해석

- 훈련 정확도와 테스트 정확도가 약 0.02정도 차이가 나면서 훈련 정확도가 테스트 정확도보다 크기 때문에 과소/과대 적합이 일어났다고 볼 수 없다.

- 훈련 정확도 0.989로 매우 훌륭한 성능을 지닌 모델로 판단된다.

- 그렇기 때문에 이 모델은 일반화된 모델로 예측이 가능하다.


In [19]:
### 테스트 데이터로 예측하기
pred = lr.predict(test_poly)
pred

array([  22.92877835,   31.10777487,  250.98436388,  111.59208216,
        128.85787135,  779.24558158,  304.72417951,  176.31471164,
        916.8961555 ,   98.7200446 , 1180.23592257,   34.86148711,
        288.45880297,  272.52503942,   85.71366627,  120.24574045,
         59.93605383])

In [20]:
### 모델 오차 확인하기 (평균절대오차(MAE))
mae = mean_absolute_error(test_target,pred)
mae

30.216889590337882

#### 해석

- 특성 공학을 적용하지 않았을 때의 다중회귀모델보다, 특성 공학을 적용한 모델이 성능이 향상되었음

- 본 모델은 degree 2를 적용한 모델로, 약 30.2 정도의 예측 오차를 발생하는 것으로 보인다.

### <좀 더 성능을 높일 수 있는 방법을 찾아보자>

- 정규화(표준화) 진행해보기

- 규제가 적용된 모델로 훈련 시켜보기

- 단, 성능을 높이는 방법을 적용하였다고 해서, 성능이 무조건 높아지는 것은 아님 

- 오히려 낮아질 수 있음(이런 경우에는 지금까지 수행한 모델 처리 방법 중 가장 좋은 모델을 사용하면 된다) 

#### <규제>

- 훈련 시에 좀 더 복잡도를 추가하여 집중력을 강화시키는 방법으로, 모델이 훈련할 때 복잡도를 증가/감소함으로써 과대/과소 적합을 조정하는 방식이다.

- 과대 또는 과소 적합이 발생하였을 때 주로 사용되는 방식이다.

- 훈련의 정확도가 다소 낮아지는 경향이 있으나, 검증(테스트) 정확도를 높이는 효과가 있다.

- 훈련 모델을 일반화하는데 주로 사용되는 방법이다.

- 규제 개념을 적용한 향상된 모델 : 릿지(Ridge), 라쏘(Lasso) 모델이 있다.

In [21]:
### 정규화 하기
### 현재 사용하는 훈련 및 테스트 독립변수 다시 한번 확인
train_poly.shape, test_poly.shape

((39, 9), (17, 9))

In [22]:
### 정규화를 위한 라이브러리 불러오기
from sklearn.preprocessing import StandardScaler

### <정규화(표준화) 순서>

1. 정규화(표준화) 클래스 생성하기
2. `fit()` : 정규화 패턴 찾기(훈련 독립 변수만 사용한다)
3. `transform()` : 찾은 패턴으로 데이터 변환하기(정규화) / 훈련 및 테스트 독립 변수 모두 변환한다. 

In [23]:
### 정규화 클래스 생성하기

ss = StandardScaler()
ss

StandardScaler()

In [24]:
### 정규화 패턴 찾기 -> 훈련 독립 변수만 사용한다.

ss.fit(train_poly)

StandardScaler()

In [25]:
### 찾은 패턴으로 훈련 및 테스트 독립 변수 데이터 변환하기(정규화)

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

train_scaled, test_scaled
train_scaled.shape, test_scaled.shape

((39, 9), (17, 9))

In [26]:
### 기존 모델을 이용해서 다시 한 번 훈련 시켜보기
### 모델 훈련시키기
lr.fit(train_scaled,train_target)

### 훈련 및 테스트 정확도 확인하
train_score1 = lr.score(train_scaled,train_target)
test_score1 = lr.score(test_scaled,test_target)


print(train_score, test_score)
print(train_score1, test_score1)
print(train_score - test_score)

0.9898271546307026 0.9713771600629739
0.9898271546307026 0.9713771600629604
0.018449994567728667


#### 해석

- 기존 특성 공학을 적용한 모델과 정규화 처리한 모델과의 정확도는 동일한 것으로 보인다.

#### 릿지(Ridge) 모델 사용하기

- 사용 패키지 : sklearn.linear_model

- 사용 모델명 : Ridge

In [27]:
# 라이브러리 import
from sklearn.linear_model import Ridge

### 훈련 모델 생성하기
ridge = Ridge()

### 모델 훈련시키기
ridge.fit(train_scaled,train_target)

### 훈련 및 테스트 정확도 확인하기
train_score = ridge.score(train_scaled, train_target)
test_score = ridge.score(test_scaled, test_target)

train_score, test_score, train_score - test_score

(0.9849041294689238, 0.9845173591615218, 0.0003867703074019735)

#### 해석

- 기존 모델보다 릿지 모델의 훈련 정확도는 다소 낮아 졌으나, 테스트 정확도는 높아짐

- 훈련과 테스트의 차이가 매우 줄어든 일반화된 모델로 판단됨

- 기존 모델보다 일반화는 잘 되었다고 판단한다(다만, 훈련 정확도는 다소 낮아졌음)

In [28]:
### 예측하기
pred = ridge.predict(test_scaled)
pred

array([ -61.93263901,   69.38828574,  279.34790525,  140.45052142,
        137.16491873,  796.87105093,  335.18396118,  207.78335133,
        836.88122091,  118.54861871, 1083.37235419,   26.09003783,
        294.35909149,  347.40000282,   74.06227077,  130.15404326,
         70.08272221])

In [29]:
### 평균 절대오차 확인하기
mae = mean_absolute_error(test_target, pred)
mae

29.25391147401826

#### 해석

- 기존 모델을 선택한다면
    
    - 훈련 정확도가 좀 더 높은 일반화된 모델을 선정하였음
 
- 릿지 모델을 선택한다면

    - 기존 모델보다 훈련 정확도는 낮아졌지만, 테스트 정확도가 높고, 오차가 1g 줄어든 모델.
    
    - 일반화가 매우 우수한 릿지 모델을 선정하였다.

In [30]:
### 라쏘(Lasso) 모델 사용하기

# 라이브러리 import
from sklearn.linear_model import Lasso

# 모델 생성하기
lasso = Lasso()

# 모델 훈련시키기
lasso.fit(train_scaled, train_target)

# 훈련 및 테스트 정확도 확인
train_score = lasso.score(train_scaled, train_target)
test_score = lasso.score(test_scaled, test_target)

print(train_score, test_score)
print(train_score - test_score)

0.9861305259897015 0.98632202810554
-0.0001915021158385155


#### 해석

- 기존 릿지 정확도 : (0.9849041294689238, 0.9845173591615218, 0.0003867703074019735)

- 릿지 모델에 비하여 훈련 정확도는 높아졌으며, 테스트 정확도 또한 높아졌으나 미세하게 과소적합을 보이고 있다.

- 매우 경미한 과소적합의 경우에는 사용은 가능하지만, 이를 해소하지 않고서 사용하기에는 미흡할 것으로 여겨진다.

In [31]:
# 에측하기
pred = lasso.predict(test_scaled)
pred

array([ -43.70669743,   70.41454264,  274.03291416,  138.14023972,
        136.1833532 ,  795.22130424,  328.60470097,  204.24315928,
        822.62808732,  118.11124527, 1075.40813513,   28.64520876,
        292.98922415,  341.90442243,   75.48401454,  129.1560788 ,
         71.01982925])

In [32]:
# 평균절대오차(MAE) 확인하기
mae = mean_absolute_error(test_target, pred)
mae

26.878384450173368

### 릿지와 라쏘 모델에 규제 적용하기

- 규제를 하이퍼 파라미터라고 한다.

- 규제 속성(변수)명 : alpha

- 규제 적용 시점 : 모델(클래스) 생성 시에 속성값으로 넣어준다.

- alpha(규제)의 기본(default) 값 = 1

- alpha의 값의 법위 : 0.001 ~ 100 사이의 값 중 한 개를 사용한다.

- 규제 해석

    - alpha 값이 작을수록 : 훈련 정확도는 낮아지면서, 과적합에 도움을 줄 수 있음
 
    - alpha 값이 클수록 : 훈련 정확도는 높아지지만, 과적합에는 도움이 안 될 수도 있음

In [33]:
### 모델 생성 시 규제 값 적용하기
ridge = Ridge(alpha=0.1)
ridge

Ridge(alpha=0.1)

In [34]:
### 모델 훈련시키기
ridge.fit(train_scaled,train_target)

### 훈련 및 테스트 정확도 확인하기
train_score = ridge.score(train_scaled, train_target)
test_score = ridge.score(test_scaled, test_target)

train_score, test_score, train_score - test_score

# - alpha = 1일 때 정확도 : (0.9849041294689238, 0.9845173591615218, 0.0003867703074019735)

# - alpha = 0.1일 때 정확도 : (0.9882780161390031, 0.9868237771849514, 0.001454238954051723)

(0.9882780161390031, 0.9868237771849514, 0.001454238954051723)

In [35]:
#### 예측하기
pred = ridge.predict(test_scaled)

### 평균 절대오차 확인하기
mae = mean_absolute_error(test_target, pred)
mae

20.23382804588946

#### 해석

- 기존 alpha값 1을 사용했을 때보다 훈련의 성능은 높아졌으나, 일반화는 다소 낮아졌음

- 다만, 그 차이는 매우 경미하기에 예측에 따른 MAE를 확인한 결과 alpha값 1을 사용했을 때보다 0.1을 사용했을 때의 모델이 매우 월등하게 오차를 줄일 수 있는 모델로 여겨진다.

- 따라서, 본 데이터를 이용한 예측 모델로는 특성공학을 적용한 degree값 2와, 정규화를 거친, alpha값 0.1의 Ridge 모델을 선정하는 것이 타당하다고 판단된다.

In [37]:
### Lasso 모델 생성하기
lasso = Lasso(alpha=0.1)

### 모델 훈련시키기
lasso.fit(train_scaled,train_target)

### 훈련 정확도와 테스트 정확도 확인하기
train_score = lasso.score(train_scaled,train_target)
test_score = lasso.score(test_scaled,test_target)

print(train_score, test_score, train_score - test_score)

test_pred = lasso.predict(test_scaled)
mae = mean_absolute_error(test_target,test_pred)
print(mae)


### 릿지 -> alpha = 0.1일 때 정확도 : (0.9882780161390031, 0.9868237771849514, 0.001454238954051723) / mae => 20.23382804588946

0.9883448062768178 0.9857016019582313 0.002643204318586445
21.295562116910816


C:\ProgramData\anaconda3\envs\pk_base\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.443e+03, tolerance: 4.887e+02
  model = cd_fast.enet_coordinate_descent(
